Required inputs for Akita are:
* binned Hi-C or Micro-C data stored in cooler format (https://github.com/mirnylab/cooler)
* Genome FASTA file

First, make sure you have a FASTA file available consistent with genome used for the coolers. Either add a symlink for a the data directory or download the machine learning friendly simplified version in the next cell.

In [1]:
import json
import os
import shutil
import subprocess

In [2]:
if not os.path.isfile('./data/hg38.ml.fa'):
    print('downloading hg38.ml.fa')
    subprocess.call('curl -o ./data/hg38.ml.fa.gz https://storage.googleapis.com/basenji_barnyard2/hg38.ml.fa.gz', shell=True)
    subprocess.call('gunzip ./data/hg38.ml.fa.gz', shell=True)

Download a few Micro-C datasets, processed using distiller (https://github.com/mirnylab/distiller-nf), binned to 2048bp, and iteratively corrected. 

In [3]:
if not os.path.exists('./data/coolers'):
    os.mkdir('./data/coolers')
if not os.path.isfile('./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool'):
    subprocess.call('curl -o ./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool'+
            ' https://storage.googleapis.com/basenji_hic/tutorials/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool', shell=True)
    subprocess.call('curl -o ./data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool'+
            ' https://storage.googleapis.com/basenji_hic/tutorials/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool', shell=True)

In [4]:
ls ./data/coolers/

H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool
HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool


Write out these cooler files and labels to a samples table.

In [5]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'HFF', './data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool', '2', 'sum', 'HFF'])
lines.append(['1', 'H1hESC', './data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool', '2', 'sum', 'H1hESC'])

samples_out = open('data/microc_cools.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

Next, we want to choose genomic sequences to form batches for stochastic gradient descent, divide them into training/validation/test sets, and construct TFRecords to provide to downstream programs.

The script [akita_data.py](https://github.com/calico/basenji/blob/master/bin/akita_data.py) implements this procedure.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --sample | 0.1 | Down-sample the genome to 10% to speed things up here. |
| -g | data/hg38_gaps_binsize2048_numconseq10.bed | Dodge large-scale unmappable regions determined from filtered cooler bins. |
| -l | 1048576 | Sequence length. |
| --crop | 65536 | Crop edges of matrix so loss is only computed over the central region. |
| --local | True | Run locally, as opposed to on a SLURM scheduler. |
| -o | data/1m | Output directory |
| -p | 8 | Uses multiple concourrent processes to read/write. |
| -t | .1 | Hold out 10% sequences for testing. |
| -v | .1 | Hold out 10% sequences for validation. |
| -w | 2048 | Pool the nucleotide-resolution values to 2048 bp bins. |
| fasta_file| data/hg38.ml.fa | FASTA file to extract sequences from. |
| targets_file | data/microc_cools.txt | Target table with cooler paths. |

Note: make sure to export BASENJIDIR as outlined in the basenji installation tips 
(https://github.com/calico/basenji/tree/master/#installation). 

In [29]:
if os.path.isdir('data/1m'):
    shutil.rmtree('data/1m')

In [30]:
! akita_data.py --sample 0.05 -g ./data/hg38_gaps_binsize2048_numconseq10.bed -l 1048576 --crop 65536 --local -o ./data/1m --as_obsexp -p 8 -t .1 -v .1 -w 2048 --snap 2048 --stride_train 262144 --stride_test 32768 ./data/hg38.ml.fa ./data/microc_cools.txt

Contigs divided into
 Train:   413 contigs, 2078450861 nt (0.8036)
 Valid:    47 contigs,  254228224 nt (0.0983)
 Test:     48 contigs,  253678336 nt (0.0981)
writing sequences to BED
akita_data_read.py --crop 65536 -d 2 -k 0 -w 2048 --clip 2.000000 --as_obsexp ./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool ./data/1m/sequences.bed ./data/1m/seqs_cov/0.h5
akita_data_read.py --crop 65536 -d 2 -k 0 -w 2048 --clip 2.000000 --as_obsexp ./data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool ./data/1m/sequences.bed ./data/1m/seqs_cov/1.h5
/opt/anaconda3/lib/python3.11/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in divide
  val_cur = ar_cur / armask_cur
/opt/anaconda3/lib/python3.11/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in divide
  val_cur = ar_cur / armask_cur
/opt/anaconda3/lib/python3.11/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in divide
  v

The data for training is now saved in data/1m as tfrecords (for training, validation, and testing), where *contigs.bed* contains the original large contiguous regions from which training sequences were taken, and *sequences.bed* contains the train/valid/test sequences.

In [31]:
! cut -f4 data/1m/sequences.bed | sort | uniq -c

 314 test
 350 train
 317 valid


In [32]:
! head -n3 data/1m/sequences.bed

chr13	44703744	45752320	train
chr3	143521792	144570368	train
chr6	130879488	131928064	train


Now train a model!

(Note: for training production-level models, please remove the --sample option when generating tfrecords)

In [33]:
# specify model parameters json to have only two targets
params_file   = './params.json'
with open(params_file) as params_file:
    params_tutorial = json.load(params_file)   
params_tutorial['model']['head_hic'][-1]['units'] =2
with open('./data/1m/params_tutorial.json','w') as params_tutorial_file:
    json.dump(params_tutorial,params_tutorial_file) 
    
### note that training with default parameters requires GPU with >12Gb RAM ###

In [ ]:
! akita_train.py -k -o ./data/1m/train_out/  ./data/1m/params_tutorial.json ./data/1m/

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 sequence (InputLayer)       [(None, 1048576, 4)]         0         []                            
                                                                                                  
 stochastic_reverse_complem  ((None, 1048576, 4),         0         ['sequence[0][0]']            
 ent (StochasticReverseComp   ())                                                                 
 lement)                                                                                          
                                                                                                  
 stochastic_shift (Stochast  (None, 1048576, 4)           0         ['stochastic_reverse_complemen
 icShift)                                                           t[0][0]']               

See explore_model.ipynb for tips on investigating the output of a trained model. 